In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda:0')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
path = 'C:/Users/ideal/Downloads/jupyter/user_data/'

In [4]:
train_png = sorted(glob(path + 'train/*.png'))
test_png = sorted(glob(path + 'test/*.png'))

In [5]:
len(train_png), len(test_png)

(858, 215)

In [6]:
train_y = pd.read_csv(path +"train.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
#     img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [8]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|███████████████████████████████████████████████████████████████████████████████| 215/215 [00:00<00:00, 934.95it/s]


In [9]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [10]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [11]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('densenet121', pretrained=True)
        if self.mode == 'test':
          self.model = timm.create_model('densenet121', pretrained=True)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [13]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)
batch_size = 8
epochs = 100
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-3)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/100    time : 31s/3077s
TRAIN    loss : 4.35489    f1 : 0.01822
Val    loss : 1.65452    f1 : 0.60067
-----------------SAVE:2 epoch----------------
epoch : 2/100    time : 29s/2854s
TRAIN    loss : 1.12157    f1 : 0.68459
Val    loss : 0.76054    f1 : 0.79702
-----------------SAVE:3 epoch----------------
epoch : 3/100    time : 30s/2902s
TRAIN    loss : 0.68063    f1 : 0.79297
Val    loss : 0.48298    f1 : 0.84864
-----------------SAVE:4 epoch----------------
epoch : 4/100    time : 29s/2819s
TRAIN    loss : 0.48256    f1 : 0.87138
Val    loss : 0.30362    f1 : 0.90505
epoch : 5/100    time : 30s/2807s
TRAIN    loss : 0.33780    f1 : 0.90997
Val    loss : 0.33350    f1 : 0.88822
-----------------SAVE:6 epoch----------------
epoch : 6/100    time : 29s/2706s
TRAIN    loss : 0.30725    f1 : 0.90521
Val    loss : 0.22649    f1 : 0.92196
-----------------SAVE:7 epoch----------------
epoch : 7/100    t

epoch : 7/100    time : 29s/2730s
TRAIN    loss : 0.22213    f1 : 0.93939
Val    loss : 0.17349    f1 : 0.94683
epoch : 8/100    time : 29s/2651s
TRAIN    loss : 0.16557    f1 : 0.96022
Val    loss : 0.18058    f1 : 0.94063
-----------------SAVE:9 epoch----------------
epoch : 9/100    time : 29s/2643s
TRAIN    loss : 0.18521    f1 : 0.95607
Val    loss : 0.17114    f1 : 0.96555
epoch : 10/100    time : 28s/2516s
TRAIN    loss : 0.14681    f1 : 0.96002
Val    loss : 0.14922    f1 : 0.94681
epoch : 11/100    time : 29s/2570s
TRAIN    loss : 0.15767    f1 : 0.95168
Val    loss : 0.15878    f1 : 0.95384
epoch : 12/100    time : 29s/2591s
TRAIN    loss : 0.12535    f1 : 0.96354
Val    loss : 0.12888    f1 : 0.96538
-----------------SAVE:13 epoch----------------
epoch : 13/100    time : 29s/2548s
TRAIN    loss : 0.12163    f1 : 0.97251
Val    loss : 0.09764    f1 : 0.97658
epoch : 14/100    time : 29s/2536s
TRAIN    loss : 0.11303    f1 : 0.97149
Val    loss : 0.14182    f1 : 0.95247
epoch 

epoch : 33/100    time : 29s/1944s
TRAIN    loss : 0.07556    f1 : 0.97860
Val    loss : 0.26948    f1 : 0.93150
epoch : 34/100    time : 29s/1909s
TRAIN    loss : 0.04446    f1 : 0.99157
Val    loss : 0.21077    f1 : 0.94322
epoch : 35/100    time : 28s/1849s
TRAIN    loss : 0.04131    f1 : 0.98719
Val    loss : 0.19508    f1 : 0.94870
epoch : 36/100    time : 29s/1870s
TRAIN    loss : 0.06002    f1 : 0.97734
Val    loss : 0.21825    f1 : 0.94309
epoch : 37/100    time : 30s/1881s
TRAIN    loss : 0.05143    f1 : 0.98479
Val    loss : 0.27390    f1 : 0.93771
epoch : 38/100    time : 29s/1808s
TRAIN    loss : 0.02612    f1 : 0.99572
Val    loss : 0.27853    f1 : 0.93786
epoch : 39/100    time : 30s/1802s
TRAIN    loss : 0.02339    f1 : 0.99575
Val    loss : 0.24305    f1 : 0.93143
epoch : 40/100    time : 29s/1734s
TRAIN    loss : 0.01759    f1 : 0.99725
Val    loss : 0.20796    f1 : 0.95513
epoch : 41/100    time : 27s/1610s
TRAIN    loss : 0.01067    f1 : 0.99862
Val    loss : 0.22734

-----------------SAVE:12 epoch----------------
epoch : 12/100    time : 29s/2592s
TRAIN    loss : 0.17625    f1 : 0.95039
Val    loss : 0.16164    f1 : 0.96600
epoch : 13/100    time : 29s/2556s
TRAIN    loss : 0.10870    f1 : 0.97356
Val    loss : 0.18966    f1 : 0.93666
epoch : 14/100    time : 30s/2609s
TRAIN    loss : 0.10287    f1 : 0.97981
Val    loss : 0.27446    f1 : 0.93844
epoch : 15/100    time : 30s/2515s
TRAIN    loss : 0.07419    f1 : 0.99005
Val    loss : 0.21380    f1 : 0.94928
epoch : 16/100    time : 30s/2502s
TRAIN    loss : 0.07023    f1 : 0.98310
Val    loss : 0.18339    f1 : 0.94680
epoch : 17/100    time : 30s/2463s
TRAIN    loss : 0.07069    f1 : 0.98151
Val    loss : 0.19370    f1 : 0.94924
epoch : 18/100    time : 30s/2428s
TRAIN    loss : 0.03559    f1 : 0.99859
Val    loss : 0.13443    f1 : 0.95346
epoch : 19/100    time : 29s/2358s
TRAIN    loss : 0.06537    f1 : 0.98584
Val    loss : 0.12681    f1 : 0.95871
epoch : 20/100    time : 30s/2392s
TRAIN    loss 

In [14]:
pred_ensemble = []
batch_size = 8
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [15]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [16]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

In [17]:
submission = pd.read_csv(path + "sample_submission.csv")
submission["label"] = f_result
submission

,file_name,label
0,001.png,1
1,002.png,2
2,003.png,1
3,004.png,6
4,005.png,8
...,...,...
210,211.png,5
211,212.png,8
212,213.png,3
213,214.png,6


In [18]:
submission.to_csv(path + "submit.csv", index = False)

In [19]:
# d9249@kyonggi.ac.kr

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'C:/Users/ideal/Downloads/jupyter/user_data/submit.csv', 
'02438df9bd0f6300dae6ddea845e7e01d2cb1881849c166bfce504164e1507d5', 
'235896', 
'iDeal', 
'test' )

{'isSubmitted': True, 'detail': 'Success'}
